# MNIST Search Quest I

Test MNIST data on a GPU and CPU using a Convolutional Neural Network.

Original CNN code from 

> https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py



In [7]:
# Test of StartupSequence..

def StartupSequence_EnableGPU3(gpu_mem_fraction=0.1, gpus=None, cpus=None, verbose=0):
    if (verbose>0):
        print(f"StartupSequence, gpu_mem_fraction={gpu_mem_fraction},  gpus={gpus}, cpus={cpus}\n")
    try:
        import tensorflow as tf
        from keras.backend.tensorflow_backend import set_session
        config = tf.ConfigProto()
        
        if gpus!=None or cpus!=None:
            if verbose>0:
                    print("WARNING: gpus and cpus options not tested")
            if gpus==None:
                gpus=1 # default
            if cpus==None:
                cpus=4 # default
            config = tf.ConfigProto( device_count = { 'GPU': gpus, 'CPU': cpus})
        
        if gpu_mem_fraction<0:
            raise ValueError("gpu_mem_fraction is below zero")
        if gpu_mem_fraction>0.5:
            raise ValueError("sorry Dave, can't do gpu_mem_fraction>50%")
        
        config.gpu_options.per_process_gpu_memory_fraction = gpu_mem_fraction
        config.gpu_options.allow_growth=True
        set_session(tf.Session(config=config))
        
        return config    
    except Exception as e:
        if (verbose>0):
            print("ERROR: something failed in StartupSequence_EnableGPU()\n")
        raise e
            
#c = StartupSequence_EnableGPU()
#c = StartupSequence_EnableGPU3(gpu_mem_fraction=0.1, gpus=0, verbose=1)
#print(f"c={c}")

#import tensorflow as tf
#from keras.backend.tensorflow_backend import get_session
#c = get_session()

#import sys, os
#print(sys.path, "\n")
! hostname
! nvidia-smi

node4
Wed Oct 23 13:58:53 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.39       Driver Version: 418.39       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    On   | 00000000:01:00.0 Off |                  N/A |
| 24%   39C    P8    13W / 180W |    760MiB /  8119MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
| 

In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28
input_shape = (img_rows * img_cols, 1)

# the data, split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

def Reshape(X_train, X_test):
    if K.image_data_format() == 'channels_first':
        X_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        X_test  = x_test.reshape (x_test.shape [0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
        X_test  = X_test.reshape (X_test.shape [0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)
    return input_shape, X_train, X_test

X_train = X_train.reshape(60000, 784)
X_test  = X_test.reshape (10000, 784)
    
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

input_shape, X_train, X_test = Reshape(X_train, X_test)

print(f'X_train shape={X_train.shape}')
print(f'X_test shape ={X_test.shape}')
print(f'input_shape={input_shape}')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test,  num_classes)

def Model1():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

#model=MLPClassifier(activation='relu', alpha=0.05, batch_size='auto', beta_1=0.9,
#       beta_2=0.999, early_stopping=False, epsilon=1e-08,
#       hidden_layer_sizes=(20, 50, 70, 100, 70, 50, 20),
#       learning_rate='adaptive', learning_rate_init=0.001, max_iter=500,
#       momentum=0.9,  nesterovs_momentum=True,
#       power_t=0.5, random_state=None, shuffle=True, solver='sgd',
#       tol=0.0001, validation_fraction=0.1, verbose=False,
#       warm_start=False)

def Model2():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))
    return model

model = Model1()
model.summary()
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

StartupSequence_EnableGPU(gpu_mem_fraction=0.1, gpus=1, verbose=1)
import time
tic_gpu = time.time()

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))

toc_gpu = time.time()
print('OK')

X_train shape=(60000, 28, 28, 1)
X_test shape =(10000, 28, 28, 1)
input_shape=(28, 28, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
____________________________________________________

In [4]:
elapsed_gpu = toc_gpu - tic_gpu
print(f'Time(train, GPU)={elapsed_gpu:0.1f} sec')

score_gpu = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score_gpu[0]}')
print(f'Test accuracy: {score_gpu[1]:0.8f}')

Time(train, GPU)=50.2 sec
Test loss: 0.025456326827264092
Test accuracy: 0.99220000


In [5]:
model = Model1()
model.summary()
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

StartupSequence_EnableGPU(gpu_mem_fraction=0.1, gpus=0, verbose=1)
tic_cpu = time.time()

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))

toc_cpu = time.time()
print('OK')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
__________

In [6]:
elapsed_cpu = toc_cpu - tic_cpu
print(f'Time(train, CPU)={elapsed_cpu:0.1f} sec')

score_cpu = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score_cpu[0]}')
print(f'Test accuracy: {score_cpu[1]:0.8f}')

Time(train, CPU)=956.8 sec
Test loss: 0.02914932119876762
Test accuracy: 0.99190000
